In [2]:
# mase_xinyi/machop/chop/actions/search/search_space/zero_cost/xautodl
import os
os.chdir("/home/xz2723/mase_xinyi/machop")
# /home/xz2723/mase_xinyi/machop/chop/actions/search/strategies/runners/software
from chop.actions.search.search_space.zero_cost.xautodl.models import get_cell_based_tiny_net
from chop.actions.search.search_space.zero_cost_nas.pruners.predictive import find_measures
# from xautodl.models import get_cell_based_tiny_net
from nas_201_api import NASBench201API as API

/home/xz2723/miniforge3/envs/mase/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from chop.dataset import MaseDataModule, get_dataset_info
from chop.tools.logger import set_logging_verbosity

from chop.passes.graph import (
    save_node_meta_param_interface_pass,
    report_node_meta_param_analysis_pass,
    profile_statistics_analysis_pass,
    add_common_metadata_analysis_pass,
    init_metadata_analysis_pass,
    add_software_metadata_analysis_pass,
)
from chop.tools.get_input import InputGenerator
from chop.tools.checkpoint_load import load_model
from chop.ir import MaseGraph

from chop.models import get_model_info, get_model

import torch.nn.functional as F

from chop.ir import MaseGraph

In [4]:
batch_size = 8
model_name = "vgg7"
dataset_name = "cifar10"

data_module = MaseDataModule(
    name=dataset_name,
    batch_size=batch_size,
    model_name=model_name,
    num_workers=0,
)

data_module.prepare_data()
data_module.setup()

model_info = get_model_info(model_name)

model = get_model(
    model_name,
    task="cls",
    dataset_info=data_module.dataset_info,
    pretrained=False,
)

input_generator = InputGenerator(
    data_module=data_module,
    model_info=model_info,
    task="cls",
    which_dataloader="train",
)

# a demonstration of how to feed an input value to the model
dummy_in = next(iter(input_generator))
_ = model(**dummy_in)


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [9]:
mg = MaseGraph(model=model)
mg, _ = init_metadata_analysis_pass(mg, None)
mg, _ = add_common_metadata_analysis_pass(mg, {"dummy_in": dummy_in})
mg, _ = add_software_metadata_analysis_pass(mg, None)

In [12]:
for node in mg.fx_graph.nodes:
    print(node.meta["mase"].parameters)
    break

{'common': {'mase_type': 'placeholder', 'mase_op': 'placeholder', 'args': {}, 'results': {'data_out_0': {'type': 'float', 'precision': [32], 'shape': [8, 3, 32, 32], 'torch_dtype': torch.float32, 'value': tensor([[[[ 0.0741,  0.1083,  0.0056,  ...,  1.5639,  0.7933,  0.4851],
          [ 0.0056,  0.1254,  0.0741,  ...,  0.2453,  0.7077,  0.2967],
          [ 0.0569,  0.1254,  0.1254,  ..., -0.6400,  0.3309,  0.1083],
          ...,
          [-0.3883, -0.3883, -0.3883,  ..., -0.3883, -0.3883, -0.3883],
          [-0.3883, -0.3883, -0.3883,  ..., -0.3883, -0.3883, -0.3883],
          [-0.3883, -0.3883, -0.3883,  ..., -0.3883, -0.3883, -0.3883]],

         [[ 0.4678,  0.4853,  0.3452,  ...,  0.2227,  1.2731,  0.8704],
          [ 0.4153,  0.5203,  0.4153,  ...,  0.5378,  1.0980,  0.6254],
          [ 0.5028,  0.5553,  0.5728,  ...,  0.7406,  0.6078,  0.3803],
          ...,
          [-0.2675, -0.2675, -0.2675,  ..., -0.2675, -0.2675, -0.2675],
          [-0.2675, -0.2675, -0.2675,  ...,

In [5]:
api = API('/home/xz2723/mase_xinyi/machop/third_party/NAS-Bench-201-v1_1-096897.pth', verbose=False)
print("api loaded")


api loaded


In [7]:
config = api.get_net_config(1, 'cifar10')
model_nasbench = get_cell_based_tiny_net(config)
dataload_info = ('random', 1, 10)
device = "cpu"
train_dataloader = data_module.train_dataloader()

In [8]:
iterator = iter(train_dataloader)
data = next(train_dataloader)

TypeError: 'DataLoader' object is not an iterator

In [1]:
# network = get_cell_based_tiny_net(config)

value = find_measures(model_nasbench, 
                    train_dataloader,
                    dataload_info, # a tuple with (dataload_type = {random, grasp}, number_of_batches_for_random_or_images_per_class_for_grasp, number of classes)
                    device, 
                    loss_fn=F.cross_entropy, 
                    measure_names='grad_norm',
                    measures_arr=None)

NameError: name 'find_measures' is not defined

In [7]:
def generate_configs(config_dict):

    name = config_dict['name'] # Assuming only one name option
    C = config_dict['C']  # Assuming only one C option
    N = config_dict['N']  # Assuming only one N option
    number_classes = config_dict['number_classes']  # Assuming only one number_classes option

    op_map = {0:'skip_connect', 1:'none', 2:'nor_conv_3x3', 3:'nor_conv_1x1', 4:'avg_pool_3x3'}

    ### generate combination
    arch_str = ""
    for target_neuro in range(1, 4):
        arch_str += "|"
        for exert_neuro in range(0, target_neuro):
            op = f"op_{target_neuro}_{exert_neuro}"
            op_str = op_map[config[op]]
            op_str += f"~{exert_neuro}"
            arch_str += (op_str + "|")
        if target_neuro < 3:
            arch_str += "+"
    
    config = {
        'name': name,
        'C': C,
        'N': N,
        'arch_str': arch_str,
        'num_classes': num_classes
    }

    return config

a = {'nas_zero_cost': {'name': 'infer.tiny', 'C': 16, 'N': 5, 'op_0_0': 0, 'op_1_0': 0, 'op_2_1': 1, 'op_2_2': 1, 'op_3_1': 0, 'op_3_2': 0, 'op_3_3': 0, 'number_classes': 10, 'op_2_0': 2, 'op_3_0': 2}}
generate_configs(a["nas_zero_cost"])

UnboundLocalError: cannot access local variable 'config' where it is not associated with a value